In [1]:
import gzip
import os
import shutil

import requests


def download_and_extract_dataset(url, save_path, folder_path):
    """Download and extract dataset if it doesn't exist."""
    if not os.path.exists(save_path):
        print(f"Downloading {os.path.basename(save_path)}...")
        response = requests.get(url)
        with open(save_path, "wb") as file:
            file.write(response.content)

        decompressed_file_name = os.path.splitext(os.path.basename(save_path))[0]
        decompressed_file_path = os.path.join(folder_path, decompressed_file_name)

        with gzip.open(save_path, "rb") as f_in:
            with open(decompressed_file_path, "wb") as f_out:
                shutil.copyfileobj(f_in, f_out)

        print(f"{decompressed_file_name} downloaded and extracted.")
    else:
        print(f"{os.path.basename(save_path)} already exists.")


file_info = [
    (
        "http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz",
        "train-images-idx3-ubyte.gz",
    ),
    (
        "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz",
        "train-labels-idx1-ubyte.gz",
    ),
    (
        "http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz",
        "t10k-images-idx3-ubyte.gz",
    ),
    (
        "http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz",
        "t10k-labels-idx1-ubyte.gz",
    ),
]

folder_name = "tmp/mnist"
folder_path = os.path.join(os.getcwd(), folder_name)

os.makedirs(folder_path, exist_ok=True)  # Create folder if it doesn't exist

# Download and extract each file
for url, file_name in file_info:
    path_to_save = os.path.join(folder_path, file_name)
    download_and_extract_dataset(url, path_to_save, folder_path)

# %%
import numpy as np


def read_idx3_ubyte_image_file(filename):
    """Read IDX3-ubyte formatted image data."""
    with open(filename, "rb") as f:
        magic_num = int.from_bytes(f.read(4), byteorder="big")
        num_images = int.from_bytes(f.read(4), byteorder="big")
        num_rows = int.from_bytes(f.read(4), byteorder="big")
        num_cols = int.from_bytes(f.read(4), byteorder="big")

        if magic_num != 2051:
            raise ValueError(f"Invalid magic number: {magic_num}")

        images = np.zeros((num_images, num_rows, num_cols), dtype=np.uint8)

        for i in range(num_images):
            for r in range(num_rows):
                for c in range(num_cols):
                    pixel = int.from_bytes(f.read(1), byteorder="big")
                    images[i, r, c] = pixel

    return images


def read_idx1_ubyte_label_file(filename):
    """Read IDX1-ubyte formatted label data."""
    with open(filename, "rb") as f:
        magic_num = int.from_bytes(f.read(4), byteorder="big")
        num_labels = int.from_bytes(f.read(4), byteorder="big")

        if magic_num != 2049:
            raise ValueError(f"Invalid magic number: {magic_num}")

        labels = np.zeros(num_labels, dtype=np.uint8)

        for i in range(num_labels):
            labels[i] = int.from_bytes(f.read(1), byteorder="big")

    return labels


# Example usage
folder_path = os.path.join(
    os.getcwd(), folder_name
)  # Adjust this path to where you stored the files

train_images = read_idx3_ubyte_image_file(
    os.path.join(folder_path, "train-images-idx3-ubyte")
)
train_labels = read_idx1_ubyte_label_file(
    os.path.join(folder_path, "train-labels-idx1-ubyte")
)
test_images = read_idx3_ubyte_image_file(
    os.path.join(folder_path, "t10k-images-idx3-ubyte")
)
test_labels = read_idx1_ubyte_label_file(
    os.path.join(folder_path, "t10k-labels-idx1-ubyte")
)

print(
    f"Shape of train_images: {train_images.shape}"
)  # Should output "Shape of train_images: (60000, 28, 28)"
print(
    f"Shape of train_labels: {train_labels.shape}"
)  # Should output "Shape of train_labels: (60000,)"
print(
    f"Shape of test_images: {test_images.shape}"
)  # Should output "Shape of test_images: (10000, 28, 28)"
print(
    f"Shape of test_labels: {test_labels.shape}"
)  # Should output "Shape of test_labels: (10000,)"

# %%
# Reshape the datasets from 3D to 2D
train_images_2d = train_images.reshape(
    train_images.shape[0], -1
)  # -1 infers the size from the remaining dimensions
test_images_2d = test_images.reshape(test_images.shape[0], -1)

train-images-idx3-ubyte.gz already exists.
train-labels-idx1-ubyte.gz already exists.
t10k-images-idx3-ubyte.gz already exists.
t10k-labels-idx1-ubyte.gz already exists.
Shape of train_images: (60000, 28, 28)
Shape of train_labels: (60000,)
Shape of test_images: (10000, 28, 28)
Shape of test_labels: (10000,)


In [2]:
from sklearn.tree import DecisionTreeClassifier

# Create and train a decision tree classifier
clf = DecisionTreeClassifier(max_depth=10, random_state=0)
clf.fit(train_images_2d, train_labels)

DecisionTreeClassifier(max_depth=10, random_state=0)

In [3]:
num_test_samples = len(test_images_2d)
python_predictions = clf.predict(test_images_2d)
python_accuracy = np.sum(python_predictions == test_labels) / num_test_samples
print(f"Python accuracy: {100*python_accuracy} %")

Python accuracy: 86.57000000000001 %


## Try out an SVM

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_images_2d)
test_features_normalized = scaler.transform(test_images_2d)

In [5]:
from sklearn.svm import SVC

clf = SVC(kernel="rbf", random_state=0, C=1000)
clf.fit(train_features_normalized, train_labels)

SVC(C=1000, random_state=0)

In [6]:
num_test_samples = len(test_features_normalized)
python_predictions = clf.predict(test_features_normalized)
python_accuracy = np.sum(python_predictions == test_labels) / num_test_samples
print(f"Python accuracy: {100*python_accuracy} %")

print("Number of support vectors", clf.n_support_)
print("Total number of support vectors", sum(clf.n_support_))

Python accuracy: 97.23 %
Number of support vectors [ 882  568 1919 1746 1340 1632 1310 1701 1748 1699]
Total number of support vectors 14545


## Try out an MLP neural network

In [55]:
from sklearn.neural_network import MLPClassifier

# Define and initialize the MLP classifier
mlp_large = MLPClassifier(
    hidden_layer_sizes=(397,), max_iter=30, alpha=0.0001, solver="adam", random_state=0
)
mlp_large.fit(train_features_normalized, train_labels)

MLPClassifier(hidden_layer_sizes=(397,), max_iter=30, random_state=0)

In [56]:
# Evaluate the classifier
accuracy = mlp_large.score(test_features_normalized, test_labels)
print("Accuracy:", accuracy)

layers_sizes = [mlp_large.coefs_[0].shape[0]] + [
    coef.shape[1] for coef in mlp_large.coefs_
]

print("Number of neurons per layer:", layers_sizes)

Accuracy: 0.9792
Number of neurons per layer: [784, 397, 10]


## Try out a smaller network

In [41]:
# Define and initialize the MLP classifier
clf = MLPClassifier(
    hidden_layer_sizes=(24,), max_iter=30, alpha=0.0001, solver="adam", random_state=0
)
clf.fit(train_features_normalized, train_labels)

/Users/kp/miniconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(24,), max_iter=30, random_state=0)

In [42]:
# Evaluate the classifier
accuracy = clf.score(test_features_normalized, test_labels)
print("Accuracy:", accuracy)

layers_sizes = [clf.coefs_[0].shape[0]] + [coef.shape[1] for coef in clf.coefs_]

print("Number of neurons per layer:", layers_sizes)

Accuracy: 0.9588
Number of neurons per layer: [784, 24, 10]


## Try out pruning

In [50]:
import copy

pruned_network = copy.deepcopy(clf)

In [51]:
num_zeros_before = sum(np.sum(layer == 0) for layer in pruned_network.coefs_)
print("Zeros before pruning:", num_zeros_before)

Zeros before pruning: 0


In [52]:
threshold = 1e-1

num_weights = 0
num_changed_weights = 0

for i in range(len(pruned_network.coefs_)):
    flattened_weights = pruned_network.coefs_[i].ravel()
    for j, weight in enumerate(flattened_weights):
        if abs(weight) < threshold:
            flattened_weights[j] = 0
            num_changed_weights += 1
        num_weights += 1

    # Reshape back to the original shape
    pruned_network.coefs_[i] = flattened_weights.reshape(pruned_network.coefs_[i].shape)

print(f"Number of parameters: {num_weights}")
print(f"Number of changed parameters: {num_changed_weights}")

Number of weights: 19056
Number of changed weights: 11393


In [53]:
num_zeros_after = sum(np.sum(layer == 0) for layer in pruned_network.coefs_)
print("Zeros after pruning:", num_zeros_after)

Zeros after pruning: 11393


In [54]:
# Evaluate the classifier
accuracy = pruned_network.score(test_features_normalized, test_labels)
print("Accuracy:", accuracy)

layers_sizes = [pruned_network.coefs_[0].shape[0]] + [
    coef.shape[1] for coef in pruned_network.coefs_
]

print("Number of neurons per layer:", layers_sizes)

Accuracy: 0.9508
Number of neurons per layer: [784, 24, 10]


## Prune the larger network

In [57]:
pruned_network = copy.deepcopy(mlp_large)

In [58]:
num_zeros_before = sum(np.sum(layer == 0) for layer in pruned_network.coefs_)
print("Zeros before pruning:", num_zeros_before)

Zeros before pruning: 0


In [59]:
threshold = 1e-1

num_weights = 0
num_changed_weights = 0

for i in range(len(pruned_network.coefs_)):
    flattened_weights = pruned_network.coefs_[i].ravel()
    for j, weight in enumerate(flattened_weights):
        if abs(weight) < threshold:
            flattened_weights[j] = 0
            num_changed_weights += 1
        num_weights += 1

    # Reshape back to the original shape
    pruned_network.coefs_[i] = flattened_weights.reshape(pruned_network.coefs_[i].shape)

print(f"Number of parameters: {num_weights}")
print(f"Number of changed parameters: {num_changed_weights}")

Number of parameters: 315218
Number of changed parameters: 270653


In [60]:
num_zeros_after = sum(np.sum(layer == 0) for layer in pruned_network.coefs_)
print("Zeros after pruning:", num_zeros_after)

Zeros after pruning: 270653


In [61]:
# Evaluate the classifier
accuracy = pruned_network.score(test_features_normalized, test_labels)
print("Accuracy:", accuracy)

layers_sizes = [pruned_network.coefs_[0].shape[0]] + [
    coef.shape[1] for coef in pruned_network.coefs_
]

print("Number of neurons per layer:", layers_sizes)

Accuracy: 0.9673
Number of neurons per layer: [784, 397, 10]
